<a href="https://colab.research.google.com/github/diwert-ai/First-step-in-NLP-HSE-bootcamp-/blob/main/kaggle%20competition/Logistic_Regression_%2B_re%2C_stopwords%2C_etc_%5BLB_0_78838%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount google.drive and copy dataset to colab linux instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
INPUT_DIR ='/content/drive/MyDrive/kaggle/fsnlp_hse/input/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/fsnlp_hse/output/'

WORK_DIR='/content/'
EXPERIMENT='logreg_ext'
EXPERIMENT_DIR = OUTPUT_DIR + EXPERIMENT+'/'

DATASET_DIR = INPUT_DIR
METADATA_DIR = INPUT_DIR

!mkdir {EXPERIMENT_DIR}

In [ ]:
!mkdir {EXPERIMENT_DIR}

mkdir: cannot create directory ‘/content/drive/MyDrive/kaggle/fsnlp_hse/output/logreg/’: File exists


# Install and set up Kaggle API

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving stop_words_russian.txt to stop_words_russian.txt


In [ ]:
!pip install -q kaggle --upgrade
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!kaggle competitions download -c hse-nlp-bootcamp

 90% 54.0M/59.8M [00:00<00:00, 149MB/s]
100% 59.8M/59.8M [00:00<00:00, 141MB/s]


In [ ]:
!unzip hse-nlp-bootcamp.zip

Archive:  hse-nlp-bootcamp.zip
  inflating: new_test_ml.csv         
  inflating: sample_submission.csv   
  inflating: train_ml.csv            


In [ ]:
!ls

drive		      new_test_ml.csv	     stop_words_russian.txt
hse-nlp-bootcamp.zip  sample_data	     train_ml.csv
kaggle.json	      sample_submission.csv


# Imports

In [ ]:
import pandas as pd
import numpy as np
import random
import pprint as pp
import os
import re

# from pymorphy2 import MorphAnalyzer
# from functools import lru_cache
# from multiprocessing import Pool
# from tqdm import tqdm

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, f1_score
from joblib import dump, load

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Configuration

In [ ]:
class Config:
  seed = 12536271
  samples_num = 51476
  experiment = EXPERIMENT
  strong_preprocessing = True

# Main pipeline starts here

In [ ]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(Config.seed)

## Load and process train data

In [ ]:
df = pd.read_csv('/content/train_ml.csv')
banks_list = list(df.bank.unique())
df['bank_id'] = df['bank'].apply(lambda x: banks_list.index(x))
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M')
df = df.dropna()
df = df.astype({'grades': 'int32'})
df.head()

,bank,feeds,grades,date,bank_id
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1,2017-02-16 16:10:00,0
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2,2016-12-13 01:05:00,1
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2,2020-08-04 06:38:00,4
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5,2015-04-19 20:11:00,7
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2,2019-03-28 18:36:00,3


In [ ]:
len(df)

51476

In [ ]:
subset = df.sample(Config.samples_num, random_state=Config.seed)
subset['text'] = subset.feeds.apply(lambda x: x.lower())
subset.reset_index(drop=True, inplace=True)

In [ ]:
if Config.strong_preprocessing:
  with open('/content/stop_words_russian.txt', 'r') as f:
    stop_words_ru = f.read().split()
  
  stop_words_ru.append('ним')
  regex = re.compile("[А-Яа-я]+")

  def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

  def remove_stopwords(tokens, stopwords = stop_words_ru):
      return [token for token in tokens if not token in stopwords and len(token) > 3]

  def clean_text(text):
      tokens = words_only(text)
      return ' '.join(remove_stopwords(tokens))
  
  subset['text'] = subset['text'].apply(clean_text)

subset

,bank,feeds,grades,date,bank_id,text
0,fk_otkritie,"Посещал данное отделение банка, обслуживание н...",5,2020-07-23 14:33:00,1,посещал данное отделение банка обслуживание вы...
1,sovcombank,"Обратилась в отделение Совкомбанка,по адресу г...",5,2019-08-29 18:15:00,9,обратилась отделение совкомбанка адресу казань...
2,sberbank,"Ничего не предвещало беды, когда на телефон в ...",1,2015-06-29 14:33:00,12,предвещало беды телефон пришло карта маэстро с...
3,sovcombank,Долго искали где расположить деньги на депозит...,5,2019-08-29 10:21:00,9,искали расположить деньги депозит обошли банка...
4,pochtabank,Жалоба по кредитному договору 24638722 от 09.1...,2,2018-02-21 11:13:00,5,жалоба кредитному договору чистяковой светлано...
...,...,...,...,...,...,...
51471,absolutbank,В конце апреля обратились в Абсолют банк с зая...,1,2006-06-01 18:12:00,36,конце апреля обратились абсолют банк заявление...
51472,vtb,"Добрый день, более 2 лет являюсь клиентом банк...",5,2017-06-02 19:50:00,3,добрый являюсь клиентом банка персональный мен...
51473,vtb,Обратился в банк с запросом о предоставлении п...,1,2018-01-19 21:27:00,3,обратился банк запросом предоставлении платной...
51474,ubrr,Очень понравилось вежливое общение со мной Ста...,5,2019-09-28 15:28:00,0,понравилось вежливое общение старшим менеджеро...


## Split data on train and test data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(subset[['bank_id','text']], subset['grades'], test_size=0.1, random_state=Config.seed)

## Set up train pipeline

In [ ]:
def get_numeric_data(x):
    return x[['bank_id']]

def get_text_data(x):
    return x['text']

transfomer_numeric = FunctionTransformer(get_numeric_data)
transformer_text = FunctionTransformer(get_text_data)
tokenizer = None if Config.strong_preprocessing else word_tokenize

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenizer))
            ]))
         ])),
    ('clf', LogisticRegression(random_state=Config.seed,  warm_start=True, max_iter=10000))
])

In [ ]:
pp.pprint(sorted(pipeline.get_params().keys()))

In [ ]:
param_grid = {'features__text_features__vec__max_df': [1.0],
              'features__text_features__vec__min_df': [0.001],
              'clf__C': [1.0]}

kfold = StratifiedKFold(n_splits=5)
scoring = {'F1': 'f1_micro'}
refit = 'F1'

## Perform GridSearch

In [ ]:
print(f'samples: {Config.samples_num}')
lr_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=10)
lr_model.fit(x_train, y_train)
lr_best = lr_model.best_estimator_

samples: 51476
Fitting 5 folds for each of 1 candidates, totalling 5 fits


## Test best model

In [ ]:
pred = lr_best.predict(x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           1       0.75      0.98      0.85      2788
           2       0.31      0.03      0.06       582
           3       0.17      0.01      0.02       253
           4       0.25      0.01      0.03       135
           5       0.90      0.91      0.90      1390

    accuracy                           0.78      5148
   macro avg       0.47      0.39      0.37      5148
weighted avg       0.70      0.78      0.71      5148



In [ ]:
f1_score(y_test, pred, average='micro')

0.7826340326340326

In [ ]:
lr_model.best_params_

{'clf__C': 1.0,
 'features__text_features__vec__max_df': 1.0,
 'features__text_features__vec__min_df': 0.001}

In [ ]:
lr_model.cv_results_

{'mean_fit_time': array([247.97425399]),
 'std_fit_time': array([57.18220734]),
 'mean_score_time': array([2.92629294]),
 'std_score_time': array([0.49015847]),
 'param_clf__C': masked_array(data=[1.0],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_features__text_features__vec__max_df': masked_array(data=[1.0],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_features__text_features__vec__min_df': masked_array(data=[0.001],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__C': 1.0,
   'features__text_features__vec__max_df': 1.0,
   'features__text_features__vec__min_df': 0.001}],
 'split0_test_F1': array([0.78674725]),
 'split1_test_F1': array([0.7859918]),
 'split2_test_F1': array([0.7859918]),
 'split3_test_F1': array([0.7859687]),
 'split4_test_F1': array([0.78316244]),
 'mean_test_F1': array([0.7855724]),
 'std_test_F1': array([0.00124073]),
 'rank_test_

In [ ]:
lr_model.best_score_

0.7855723965279234

In [ ]:
cv_tag = f'[cv_{lr_model.best_score_:0.4f}]'

In [ ]:
cv_tag

'[cv_0.7856]'

## Inference (calculating submission) 

In [ ]:
test = pd.read_csv('/content/new_test_ml.csv', index_col=0)
test['text'] = test.feeds.apply(lambda x: x.lower())
test['bank_id'] = test['bank'].apply(lambda x: banks_list.index(x))
if Config.strong_preprocessing:
  test['text'] = test.text.apply(clean_text)
test = test[['bank_id', 'text']]
test_pred = lr_best.predict(test)
probs = lr_best.predict_proba(test)
sub = pd.DataFrame({'inds': test.index,
                    'grades': test_pred})
sub

,inds,grades
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
17215,17215,1
17216,17216,1
17217,17217,1
17218,17218,1


# Save results to files

In [ ]:
suffix = f'{Config.experiment}_[sn_{Config.samples_num}]_[sd_{Config.seed}]'+cv_tag
sub_filename = 'sub_'+suffix+'.csv'
probs_filename = 'probs_'+suffix+'.npy'
model_filename = 'model_'+suffix+'.joblib'

In [ ]:
sub.to_csv(sub_filename, index=False)
np.save(probs_filename, probs)
dump(lr_best, model_filename) 

['model_logreg_ext_[sn_51476]_[sd_12536271][cv_0.7856].joblib']

In [ ]:
!cp {sub_filename} {EXPERIMENT_DIR}
!cp {probs_filename} {EXPERIMENT_DIR}
!cp {model_filename} {EXPERIMENT_DIR}

# Submit predictions

In [ ]:
!kaggle competitions submit -c hse-nlp-bootcamp  -f {'/content/'+sub_filename} -m "Submission via API"

100% 124k/124k [00:01<00:00, 66.9kB/s]
Successfully submitted to HSE | NLP Bootcamp | Sentiment Analysis

# Submit CV 0.7856 - public LB 0.78838